In [1]:
import pandas as pd
import re

In [95]:
draws = pd.read_csv('eoi.csv')
draws['date'] = draws.date.astype('datetime64[ns]')
draws = draws.sort_values(by=['date'], ascending=False).reset_index(drop=True)
draws = draws.reset_index()
draws.columns = ['draw_id', 'date', 'draw_type', 'score', 'invitations', 'nocs']
draws.draw_id = draws.draw_id + 1

In [96]:
nocs_draws_obj = {'noc_id': [], 'draw_id': []}
for i in range (len(draws)):
    nocs_list =  [x.strip() for x in draws.iloc[i].nocs.split(',')]
    draw_id = draws.iloc[i].draw_id
    nocs_draws_obj['noc_id'] = nocs_draws_obj['noc_id'] + nocs_list
    nocs_draws_obj['draw_id'] = nocs_draws_obj['draw_id'] + [draw_id]*len(nocs_list)

In [97]:
df = pd.read_csv('./noc_list_raw.csv')
df['noc'] = df['noc'].astype('str')
df['noc'] = df['noc'].apply(lambda x: '{0:0>4}'.format(x))
pd.set_option('display.max_rows',200)

In [98]:
nocs = df
nocs.columns = ['noc_id', 'title', 'skill_level', '2019_est_employments', '2019_wage_est', 'job_outlook', 'job_openings', 'soft_skills']
nocs_draws= pd.DataFrame(nocs_draws_obj)

In [103]:
z = nocs.merge(nocs_draws, on='noc_id', how='left').merge(draws, how='left')[['noc_id','date', 'draw_type']]
z['draw_date_type'] = z['date'].astype('str')+'_'+z['draw_type']
z = z.drop_duplicates().sort_values(by='noc_id')
x=pd.get_dummies(z['draw_date_type'])
j=pd.concat([z,x], axis=1)
j = j.drop(['draw_date_type'], axis=1)
k = j.groupby('noc_id')
final = nocs.merge(k.sum(), on='noc_id')
final = final.drop(['soft_skills', '2019_wage_est'], axis=1)
final['total'] = final.filter(regex='2019-|2020-|2021.*').sum(axis=1)
final.update(final.filter(regex='2021-|2020-|2019-.*').replace(to_replace=[1,0],value=['YES', 'NO']))
columns = final.columns.to_list()
col_1 = columns[0:6]
col_2 = [columns[-1]]
col_3 = columns[6:-1]
col_3.reverse()
col=col_1+col_2+col_3
final = final[final.total > 0]
final[col]

,noc_id,title,skill_level,2019_est_employments,job_outlook,job_openings,total,2019-09-25_Express Entry,2019-09-25_Occupations In-Demand,2019-10-02_Express Entry,...,2020-12-17_Occupations In-Demand,2021-01-07_Express Entry,2021-01-07_Occupations In-Demand,2021-01-21_Express Entry,2021-01-21_Occupations In-Demand,2021-02-11_Express Entry,2021-02-11_Occupations In-Demand,2021-02-25_Express Entry,2021-03-11_Express Entry,2021-03-11_Occupations In-Demand
0,0011,Legislators,Mgt,450,Fair,140,0,NO,NO,NO,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
1,0012,Senior government managers and officials,Mgt,690,Good,170,0,NO,NO,NO,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
2,0013,"Senior managers - financial, communications an...",Mgt,1020,Good,240,45,YES,YES,YES,...,YES,YES,YES,YES,YES,YES,YES,NO,NO,NO
3,0014,"Senior managers - health, education, social an...",Mgt,890,Good,240,36,NO,NO,YES,...,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES
4,0015,"Senior managers - trade, broadcasting and othe...",Mgt,850,Fair,250,0,NO,NO,NO,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,9612,Labourers in metal fabrication,LHS,340,Fair,50,0,NO,NO,NO,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
433,9613,Labourers in chemical products processing and ...,LHS,160,Fair,30,0,NO,NO,NO,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
434,9614,"Labourers in wood, pulp and paper processing",LHS,190,Fair,30,0,NO,NO,NO,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
435,9617,"Labourers in food, beverage and associated pro...",LHS,1710,Fair,310,0,NO,NO,NO,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO


In [109]:
columns = final.columns.to_list()

In [121]:
col_1 = columns[0:6]
col_2 = [columns[-1]]
col_3 = columns[6:-1]
col_3.reverse()
col=col_1+col_2+col_3

In [78]:
# import seaborn as sns
# corr = final.corr()
# ax = sns.heatmap(
#     corr, 
#     vmin=-1, vmax=1, center=0,
#     cmap=sns.diverging_palette(20, 220, n=1000),
#     square=True
# )
# ax.set_xticklabels(
#     ax.get_xticklabels(),
#     rotation=45,
#     horizontalalignment='right'
# );

In [1]:
# col

In [10]:
x = """Date of Invitations to Apply
Category
Score of Lowest Ranked Candidate to Apply
Total Candidates Invited to Apply
Other Considerations
May 5, 2022
Express Entry
85
106 Invited Candidates had Educational Credential Assessments.
Not all occupations were selected. The occupations chosen and eligible for the EOI selection on May 5, 2022 have the following classification numbers: 1221, 1122, 1241, 1123, 0621, 4033, 0014, 4164, 4212, 0121, 4169, 2212, 0421, 9232, 8222, 4216, 0911, 1243, 2241, 2263, 0821, 4152, 0423, 7237, 9212, 2142, 2151, 2242, 7251, 0712, 7283, 4166, 2144, 2243, 7271, 2252, 1313, 2253, 4161, 7232, 1222
May 5, 2022
Occupations In-Demand
85
91 Invited Candidates had Educational Credential Assessments.
Not all occupations were selected. The occupations chosen and eligible for the EOI selection on May 5, 2022 have the following classification numbers:
1221, 1122, 1241, 1123, 0621, 4033, 0014, 4164, 4212, 0121, 4169, 2212, 0421, 9232, 8222, 4216, 0911, 1243, 2241, 2263, 0821, 4152, 0423, 7237, 9212, 2142, 2151, 2242, 7251, 0712,
Updated: May 5, 2022
7283, 4166, 2144, 2243, 7271, 2252, 1313, 2253, 4161, 7232, 1222
May 5, 2022
71
1 Invited Candidates whose current country of residence is Ukraine as a special measure in response to the current conflict.
March 8, 2022
Express Entry
87
39 Invited Candidates had Educational Credential Assessments.
Not all occupations were selected. The occupations chosen and eligible for the EOI selection on March 8, 2022 have the following classification numbers: 0014, 0121, 0421, 0621, 0911, 1122, 1123, 1213, 1214, 1221, 1222, 1241, 1243, 1252, 2142, 2253, 2263, 3143, 3211, 3215, 4033, 4151, 4153, 4162, 4164, 4166, 4212, 7246, 7295, 8222, 9212
March 8, 2022
Occupations In-Demand
87
43 Invited Candidates had Educational Credential Assessments.
Not all occupations were selected. The occupations chosen and eligible for the EOI selection on March 8, 2022 have the following classification numbers: 0014, 0121, 0421, 0621, 0911, 1122, 1123, 1213, 1214, 1221, 1222, 1241, 1243, 1252, 2142, 2253, 2263, 3143, 3211, 3215, 4033, 4151, 4153, 4162, 4164, 4166, 4212, 7246, 7295, 8222, 9212
March 8, 2022
Express Entry & Occupations In-Demand
61
3 Invited Candidates whose current country of residence is Ukraine as a special measure in response to the current conflict.
March 4, 2022
Express Entry
61
14 Invited Candidates whose current country of residence is Ukraine as a special measure in response to the current conflict.
March 4, 2022
Occupations In-Demand
60
22 Invited Candidates whose current country of residence is Ukraine as a special measure in response to the current conflict.
January 12, 2022
Express Entry
68
37
Invited Candidates had Educational Credential Assessments.
Not all occupations were selected. The occupations chosen and eligible for the
Updated: May 5, 2022
EOI selection on January 12, 2022 have the following classification numbers:
3211, 3234, 4151, 3216, 3143, 3215
January 12, 2022
Occupations In-Demand
68
67
Invited Candidates had Educational Credential Assessments.
Not all occupations were selected. The occupations chosen and eligible for the EOI selection on January 12, 2022 have the following classification numbers:
3211, 3234, 4151, 3216, 3143, 3215
November 18, 2021
Express Entry
65
425
Invited Candidates had Educational Credential Assessments.
Not all occupations were selected. The occupations chosen and eligible for the EOI selection on November 18, 2021 have the following classification numbers:
0711, 0016, 7242, 0712, 7294, 7231, 7281, 7243, 7201, 7371, 7203, 7235, 7283, 7272
November 18, 2021
Occupations In-Demand
65
208
Invited Candidates had Educational Credential Assessments.
Not all occupations were selected. The occupations chosen and eligible for the EOI selection on November 18, 2021 have the following classification numbers:
0711, 0016, 7242, 0712, 7294, 7231, 7281, 7243, 7201, 7371, 7203, 7235, 7283, 7272
September 27, 2021
Occupations In-Demand
71
239
Invited Candidates had Educational Credential Assessments.
Not all occupations were selected. The occupations chosen and eligible for the EOI selection on September 27, 2021 have the following classification numbers:
0014, 0016, 0121, 0421, 0423, 0621, 0651, 0711, 0712, 0821, 0911, 1122, 1123, 1214, 1221, 1222, 1224, 1241, 1243, 1251, 1252, 1313, 2142, 2144, 2151, 2212, 2231, 2241, 2242, 2244, 2252, 2253, 2254, 2262, 2263, 2264, 3143, 3211, 3215, 3234, 4152, 4153,
Updated: May 5, 2022
4156, 4161, 4162, 4164, 4166, 4167, 4169, 4212, 4216, 7231, 7237, 7241, 7294, 7295, 7321, 7371, 7384, 8222, 9212
September 27, 2021
Express Entry
71
152
Invited Candidates had Educational Credential Assessments.
Not all occupations were selected. The occupations chosen and eligible for the EOI selection on September 27, 2021 have the following classification numbers:
0014, 0016, 0121, 0421, 0423, 0621, 0651, 0711, 0712, 0821, 0911, 1122, 1123, 1214, 1221, 1222, 1224, 1241, 1243, 1251, 1252, 1313, 2142, 2144, 2151, 2212, 2231, 2241, 2242, 2244, 2252, 2253, 2254, 2262, 2263, 2264, 3143, 3211, 3215, 3234, 4152, 4153, 4156, 4161, 4162, 4164, 4166, 4167, 4169, 4212, 4216, 7231, 7237, 7241, 7294, 7295, 7321, 7371, 7384, 8222, 9212
September 8, 2021
Occupations In-Demand
66
212
Invited Candidates had Educational Credential Assessments.
Not all occupations were selected. The occupations chosen and eligible for the EOI selection on September 8, 2021 have the following classification numbers:
0911, 0821, 3211, 3234, 9232, 4151, 9212, 3216, 3215, 3143, 8255, 2225, 9226, 0822, 3122, 9231
September 8, 2021
Express Entry
66
316
Invited Candidates had Educational Credential Assessments.
Not all occupations were selected. The occupations chosen and eligible for the EOI selection on September 8, 2021 have the following classification numbers:
0911, 0821, 3211, 3234, 9232, 4151, 9212, 3216, 3215, 3143, 8255, 2225, 9226, 0822, 3122, 9231
August 19, 2021
Occupations In-Demand
70
335
Invited Candidates had Educational Credential Assessments.
Updated: May 5, 2022
Not all occupations were selected. The occupations chosen and eligible for the EOI selection on August 19, 2021 have the following classification numbers: 0014, 0016, 0121, 0421, 0423, 0621, 0651, 0711, 0821, 0911, 1122, 1123, 1214, 1221, 1222, 1224, 1241, 1243, 1252, 1313, 2131, 2142, 2144, 2151, 2212, 2225, 2231, 2232, 2241, 2242, 2244, 2252, 2263, 2264, 3211, 3215, 3216, 3234, 4152, 4153, 4161, 4162, 4164, 4166, 4212, 4216, 6316, 7205, 7235, 7237, 7242, 7243, 7245, 7246, 7251, 7271, 7281, 7321, 7322, 8222, 9211
August 19, 2021
Express Entry
71
161
Invited Candidates had Educational Credential Assessments.
Not all occupations were selected. The occupations chosen and eligible for the EOI selection on August 19, 2021 have the following classification numbers: 0014, 0016, 0121, 0132, 0421, 0621, 0711, 0712, 0821, 0911, 1122, 1123, 1213, 1221, 1222, 1224, 1241, 1252, 2131, 2142, 2143, 2151, 2232, 2241, 2252, 2253, 2263, 2264, 3143, 3211, 3215, 3234, 4151, 4153, 4161, 4164, 4166, 4212, 4216, 6316, 7242, 9212, 9232
August 5, 2021
Occupations In-Demand
73
281
Invited Candidates had Educational Credential Assessments.
Not all occupations were selected. The occupations chosen and eligible for the EOI selection on August 5, 2021 have the following classification numbers:
0013, 0014, 0016, 0114, 0121, 0421, 0423, 0621, 0651, 0711, 0712, 0821, 0911, 1121, 1122, 1123, 1221, 1222, 1224, 1241, 1243, 1252, 2131, 2142, 2144, 2151, 2212, 2223, 2225, 2231, 2232, 2241, 2242, 2243, 2244, 2252, 2253, 2254, 2261, 2263, 2264, 3211, 3215, 3234, 4152, 4153, 4156, 4161, 4162, 4164, 4165, 4166, 4169, 4212, 6316, 7202, 7231, 7236, 7237, 7241, 7312, 7321, 7381, 7384, 9232
Updated: May 5, 2022
August 5, 2021
Express Entry
73
171
Invited Candidates had Educational Credential Assessments.
Not all occupations were selected. The occupations chosen and eligible for the EOI selection on August 5, 2021 have the following classification numbers: 0013, 0014, 0016, 0114, 0121, 0421, 0423, 0621, 0651, 0711, 0712, 0821, 0911, 1121, 1122, 1123, 1221, 1222, 1224, 1241, 1243, 1252, 2131, 2142, 2144, 2151, 2212, 2223, 2225, 2231, 2232, 2241, 2242, 2243, 2244, 2252, 2253, 2254, 2261, 2263, 2264, 3211, 3215, 3234, 4152, 4153, 4156, 4161, 4162, 4164, 4165, 4166, 4169, 4212, 6316, 7202, 7231, 7236, 7237, 7241, 7312, 7321, 7381, 7384, 9232
July 21, 2021
Occupations In-Demand
77
196
Invited Candidates had Educational Credential Assessments.
Not all occupations were selected. The occupations chosen and eligible for the EOI selection on July 21, 2021 have the following classification numbers:
0013, 0014, 0016, 0114, 0121, 0132, 0421, 0423, 0621, 0651, 0711, 0712, 0821, 0911, 1121, 1122, 1123, 1214, 1215, 1221, 1222, 1224, 1241, 1243, 1252, 2131, 2142, 2144, 2151, 2212, 2223, 2225, 2231, 2232, 2241, 2242, 2253, 2262, 2263, 3143, 3211, 3234, 4152, 4156, 4161, 4162, 4164, 4165, 4166, 4167, 4169, 4212, 4216, 6316, 6344, 7231, 7237, 7241, 7243, 7246, 7294, 8255, 9232
July 21, 2021
Express Entry
77
84
Invited Candidates had Educational Credential Assessments.
Not all occupations were selected. The occupations chosen and eligible for the EOI selection on July 21, 2021 have the following classification numbers:
0013, 0014, 0016, 0114, 0121, 0132, 0421, 0423, 0621, 0651, 0711, 0712, 0821, 0911, 1121, 1122, 1123, 1214, 1215, 1221, 1222, 1224, 1241, 1243, 1252, 2131, 2142, 2144, 2151, 2212,
Updated: May 5, 2022
2223, 2225, 2231, 2232, 2241, 2242, 2253, 2262, 2263, 3143, 3211, 3234, 4152, 4156, 4161, 4162, 4164, 4165, 4166, 4167, 4169, 4212, 4216, 6316, 6344, 7231, 7237, 7241, 7243, 7246, 7294, 8255, 9232
July 8, 2021
Occupations In-Demand
80
196
Invited Candidates had Educational Credential Assessments.
Not all occupations were selected. The occupations chosen and eligible for the EOI selection on July 8, 2021 have the following classification numbers:
0013, 0014, 0016, 0114, 0121, 0124, 0421, 0423, 0621, 0651, 0711, 0712, 0821, 0822, 0911, 1121, 1122, 1123, 1213, 1214, 1215, 1221, 1222, 1224, 1241, 1243, 1252, 1313, 2131, 2132, 2151, 2212, 2232, 2241, 2242, 2253, 2262, 2263, 2264, 3211, 3234, 4033, 4152, 4162, 4165, 4166, 4169, 4212, 4216, 7236, 7237, 7241, 7242, 7244, 7251, 7294, 7321, 7371, 7384, 8221, 8222, 9212
July 8, 2021
Express Entry
80
99
Invited Candidates had Educational Credential Assessments.
Not all occupations were selected. The occupations chosen and eligible for the EOI selection on July 8, 2021 have the following classification numbers:
0013, 0014, 0016, 0114, 0121, 0124, 0421, 0423, 0621, 0651, 0711, 0712, 0821, 0822, 0911, 1121, 1122, 1123, 1213, 1214, 1215, 1221, 1222, 1224, 1241, 1243, 1252, 1313, 2131, 2132, 2151, 2212, 2232, 2241, 2242, 2253, 2262, 2263, 2264, 3211, 3234, 4033, 4152, 4162, 4165, 4166, 4169, 4212, 4216, 7236, 7237, 7241, 7242, 7244, 7251, 7294, 7321, 7371, 7384, 8221, 8222, 9212
June 15, 2021
Occupations In-Demand
82
152
Invited Candidates had Educational Credential Assessments.
The following NOCs have been included:
0013, 0014, 0016, 0124, 0421, 0423, 0621, 0711, 0712, 0821, 0911, 1121,
Updated: May 5, 2022
1122, 1123, 1214, 1215, 1221, 1222, 1224, 1241, 1242, 1243, 1252, 2131, 2144, 2151, 2212, 2225, 2231, 2232, 2241, 2242, 2244, 2253, 2263, 2264, 3143, 3211, 3234, 4033, 4152, 4153, 4156, 4161, 4162, 4164, 4165, 4166, 4212, 4216, 7231, 7237, 7246, 7384, 8221
June 15, 2021
Express Entry
82
103
Invited Candidates had Educational Credential Assessments.
The following NOCs have been included:
0013, 0014, 0016, 0124, 0421, 0423, 0621, 0711, 0712, 0821, 0911, 1121, 1122, 1123, 1214, 1215, 1221, 1222, 1224, 1241, 1242, 1243, 1252, 2131, 2144, 2151, 2212, 2225, 2231, 2232, 2241, 2242, 2244, 2253, 2263, 2264, 3143, 3211, 3234, 4033, 4152, 4153, 4156, 4161, 4162, 4164, 4165, 4166, 4212, 4216, 7231, 7237, 7246, 7384, 8221
May 25, 2021
Occupations In-Demand
65
269
Invited Candidates had Educational Credential Assessments.
The following NOCs have been included:
0711, 0712, 0714, 0731, 7201, 7202, 7203, 7205, 7231, 7232, 7234, 7237, 7242, 7243, 7245, 7246, 7271, 7281, 7282, 7283, 7291, 7294, 7301, 7302, 7304, 7305, 7311, 7312, 7313, 7321, 7322, 7333, 7335, 7371, 7381, 7384
May 6, 2021
Express Entry
70
136
Invited Candidates had Educational Credential Assessments.
The following NOCs have been included:
0014, 0016, 0121, 0131, 0421, 0423, 0651, 0711, 0712, 0714, 0731, 0821, 0911, 1122, 1222, 1224, 1242, 1243, 1252, 1253, 1313, 2131, 2151, 2211, 2212, 2223, 2231, 2232, 2241, 2242, 2243, 2253, 2263, 2264, 3143, 3211, 3212, 3215, 3223, 3234, 4033, 4152, 4156, 4161, 4162, 4164, 4165, 4166, 4169, 4212, 6316, 7202, 7231, 7237, 7245, 7246, 7251, 7281, 7313, 7371, 8222, 9212, 9232
Updated: May 5, 2022
May 6, 2021
Occupations In-Demand
70
123
Invited Candidates had Educational Credential Assessments.
The following NOCs have been included:
0014, 0016, 0121, 0131, 0421, 0423, 0651, 0711, 0712, 0714, 0731, 0821, 0911, 1122, 1222, 1224, 1242, 1243, 1252, 1253, 1313, 2131, 2151, 2211, 2212, 2223, 2231, 2232, 2241, 2242, 2243, 2253, 2263, 2264, 3143, 3211, 3212, 3215, 3223, 3234, 4033, 4152, 4156, 4161, 4162, 4164, 4165, 4166, 4169, 4212, 6316, 7202, 7231, 7237, 7245, 7246, 7251, 7281, 7313, 7371, 8222, 9212, 9232
April 22, 2021
Express Entry
70
130
Invited Candidates had Educational Credential Assessments.
The following NOCs have been included:
0014, 0016, 0121, 0131, 0132, 0421, 0651, 0711, 0712, 0714, 0731, 0821, 0822, 0911, 0912, 1122, 1222, 1224, 1225, 1242, 1243, 1252, 1313, 2131, 2151, 2211, 2231, 2232, 2241, 2242, 2252, 2253, 2263, 2264, 2272, 3211, 3212, 3215, 3223, 3234, 3237, 4033, 4152, 4161, 4162, 4164, 4165, 4212, 4216, 6316, 7202, 7244, 7246, 7271, 7301, 7305, 7321, 8231, 8255, 9212, 9232
April 22, 2021
Occupations In-Demand
70
139
Invited Candidates had Educational Credential Assessments.
The following NOCs have been included:
0014, 0016, 0121, 0131, 0132, 0421, 0651, 0711, 0712, 0714, 0731, 0821, 0822, 0911, 0912, 1122, 1222, 1224, 1225, 1242, 1243, 1252, 1313, 2131, 2151, 2211, 2231, 2232, 2241, 2242, 2252, 2253, 2263, 2264, 2272, 3211, 3212, 3215, 3223, 3234, 3237, 4033, 4152, 4161, 4162, 4164, 4165, 4212, 4216, 6316, 7202, 7244, 7246, 7271, 7301, 7305, 7321, 8231, 8255, 9212, 9232
Updated: May 5, 2022
April 8, 2021
Express Entry
70
146
Invited Candidates had Educational Credential Assessments.
The following NOCs have been included:
0014, 0016, 0121, 0131, 0132, 0212, 0421, 0423, 0651, 0711, 0712, 0714, 0731, 0821, 0911, 1122, 1222, 1224, 1225, 1242, 1243, 1252, 2131, 2144, 2151, 2211, 2212, 2232, 2241, 2253, 2263, 2282, 3211, 3212, 3216, 3217, 3223, 3234, 4033, 4152, 4156, 4161, 4162, 4164, 4165, 4166, 4169, 4212, 6222, 6316, 7237, 7242, 7246, 7251, 7322, 9213, 9226, 9232
April 8, 2021
Occupations In-Demand
70
133
Invited Candidates had Educational Credential Assessments.
The following NOCs have been included:
0014, 0016, 0121, 0131, 0132, 0212, 0421, 0423, 0651, 0711, 0712, 0714, 0731, 0821, 0911, 1122, 1222, 1224, 1225, 1242, 1243, 1252, 2131, 2144, 2151, 2211, 2212, 2232, 2241, 2253, 2263, 2282, 3211, 3212, 3216, 3217, 3223, 3234, 4033, 4152, 4156, 4161, 4162, 4164, 4165, 4166, 4169, 4212, 6222, 6316, 7237, 7242, 7246, 7251, 7322, 9213, 9226, 9232
March 24, 2021
Express Entry
70
183
Invited Candidates had Educational Credential Assessments.
The following NOCs have been included:
0014, 0016, 0121, 0131, 0132, 0212, 0421, 0423, 0711, 0712, 0714, 0731, 0821, 0911, 0912, 1122, 1211, 1214, 1222, 1224, 1225, 1242, 1243, 1252, 2121, 2131, 2148, 2151, 2211, 2212, 2222, 2232, 2241, 2242, 2243, 2252, 2253, 2263, 2282, 3211, 3212, 3215, 3217, 3234, 3237, 4033, 4151, 4152, 4153, 4161, 4162, 4164, 4165, 4166, 4169, 4212, 4215, 4216, 6222, 6316, 7202, 7205, 7231, 7237, 7242, 7251, 7294, 7311, 7321, 7333, 8222, 8255, 9212, 9213, 9226, 9231, 9232, 9241
Updated: May 5, 2022
March 24, 2021
Occupations In-Demand
70
235
Invited Candidates had Educational Credential Assessments.
The following NOCs have been included:
0014, 0016, 0121, 0131, 0132, 0212, 0421, 0423, 0711, 0712, 0714, 0731, 0821, 0911, 0912, 1122, 1211, 1214, 1222, 1224, 1225, 1242, 1243, 1252, 2121, 2131, 2148, 2151, 2211, 2212, 2222, 2232, 2241, 2242, 2243, 2252, 2253, 2263, 2282, 3211, 3212, 3215, 3217, 3234, 3237, 4033, 4151, 4152, 4153, 4161, 4162, 4164, 4165, 4166, 4169, 4212, 4215, 4216, 6222, 6316, 7202, 7205, 7231, 7237, 7242, 7251, 7294, 7311, 7321, 7333, 8222, 8255, 9212, 9213, 9226, 9231, 9232, 9241
"""

In [11]:
text = x.split("\n")

In [13]:
import pandas as pd

In [16]:
text

['Date of Invitations to Apply',
 'Category',
 'Score of Lowest Ranked Candidate to Apply',
 'Total Candidates Invited to Apply',
 'Other Considerations',
 'May 5, 2022',
 'Express Entry',
 '85',
 '106 Invited Candidates had Educational Credential Assessments.',
 'Not all occupations were selected. The occupations chosen and eligible for the EOI selection on May 5, 2022 have the following classification numbers: 1221, 1122, 1241, 1123, 0621, 4033, 0014, 4164, 4212, 0121, 4169, 2212, 0421, 9232, 8222, 4216, 0911, 1243, 2241, 2263, 0821, 4152, 0423, 7237, 9212, 2142, 2151, 2242, 7251, 0712, 7283, 4166, 2144, 2243, 7271, 2252, 1313, 2253, 4161, 7232, 1222',
 'May 5, 2022',
 'Occupations In-Demand',
 '85',
 '91 Invited Candidates had Educational Credential Assessments.',
 'Not all occupations were selected. The occupations chosen and eligible for the EOI selection on May 5, 2022 have the following classification numbers:',
 '1221, 1122, 1241, 1123, 0621, 4033, 0014, 4164, 4212, 0121, 4169,